# [Day 20](https://adventofcode.com/2020/day/20): Jurassic Jigsaw

In [1]:
import numpy as np
import re

with open("../data/20.txt", "r") as f:
    lines = "".join(l.strip() for l in f.readlines())    
    id_tile = re.findall(r"Tile (\d+):([\.|#]+)", lines)

ids = np.array([int(i) for i, _ in id_tile])

tiles = np.array([list(t) for _, t in id_tile]) == "#"
tiles = tiles.reshape(-1, 10, 10)

## Part 1

In [2]:
from itertools import groupby, starmap
from collections import defaultdict, Counter
from operator import itemgetter

N = lambda x: tuple(x[0])
S = lambda x: tuple(x[-1])
W = lambda x: tuple(x[:,0])
E = lambda x: tuple(x[:,-1])

def border(tile):
    edges = N(tile), E(tile), S(tile), W(tile)
    edges = map(frozenset, zip(edges, [e[::-1] for e in edges]))
    return enumerate(edges)

def exterior(tiles):
    d = defaultdict(set)
    for tile, edges in enumerate(map(border, tiles)):
        for i, edge in edges:
            d[edge].add((tile, i))
    cts = Counter(ti.pop() for ti in d.values() if len(ti) == 1)
    return list(starmap(extedges, groupby(sorted(cts), key=fst)))

def extedges(t, edges):
    return t, tuple(sorted(i for _, i in edges))

fst = itemgetter(0)

corners = [i for i, ext in exterior(tiles) if len(ext) == 2]
assert 21599955909991 == np.prod(ids[corners])

## Part 2

In [3]:
from itertools import accumulate, product

# Counterclockwise turns
cc = [lambda x: x,
      lambda x: x.T[::-1],
      lambda x: x[::-1, ::-1],
      lambda x: x.T[:, ::-1]]

turns = {(0,): cc[0], (0,3): cc[0],
         (1,): cc[1], (0,1): cc[1],
         (2,): cc[2], (1,2): cc[2],
         (3,): cc[3], (2,3): cc[3]}

def boundary(tiles):
    i_ext = exterior(tiles)
    bd = np.array([turns[ext](tiles[i]) for i, ext in i_ext])
    iscorner = np.array([len(ext) == 2 for _, ext in i_ext])
    interior = np.ones(len(tiles), dtype=bool)
    interior[[i for i, _ in i_ext]] = False
    return bd, iscorner, tiles[interior]

def boundaries(tiles):
    return np.array(list(arrange(tiles)))

def arrange(tiles, nw=lambda c: c):
    nwcorner = lambda corners: nonnone(map(nw, corners))
    while len(tiles):
        bd, iscorner, tiles = boundary(tiles)
        c = nwcorner(bd[iscorner])
        nw = yield from hamiltonian(graph(bd, iscorner), c)

nonnone = lambda xs: next(x for x in xs if x is not None)

def hamiltonian(g, c):
    del g[E(c)][S(c)]
    e, es, cycle = E(c), {E(c)}, [c]
    for _ in range(len(g) - 1):
        e, t = next((x, t) for x, t in g[e].items() if x not in es)
        cycle.append(t)
        es.add(e)
    yield from cycle
    return matches(n=S(cycle[1]), w=S(cycle[-1])[::-1])

def graph(bd, iscorner):
    g = defaultdict(dict)
    for tile, corner in zip(bd, iscorner):
        if corner:
            g[S(tile)].update({E(tile): tile})
            g[E(tile)].update({S(tile): tile.T})
        else:
            g[W(tile)].update({E(tile): tile})
            g[E(tile)].update({W(tile): tile[:, ::-1]})
    return g

def matches(n, w):
    def match(tile):
        nw = N(tile), W(tile)
        if nw == (n, w): return tile
        if nw == (w, n): return tile.T
    return match

def layout(tiles, m=12, l=8):
    bds = boundaries(tiles)[:, 1:-1, 1:-1]
    win = img = np.empty((m*l, m*l), dtype=bool)
    for k in range(m - 1, 0, -2):
        n, e, s, w = bds[:k], bds[k:2*k], bds[2*k:3*k], bds[3*k:4*k]
        win[:l, :-l] = np.hstack(n)
        win[:-l, -l:] = np.vstack(T(e[:, ::-1]))
        win[-l:, l:] = np.hstack(s[::-1, ::-1, ::-1])
        win[l:, :l] = np.vstack(T(w[::-1, :, ::-1]))
        win = win[l:-l, l:-l]
        bds = bds[4*k:]
    return img

T = lambda x: x.transpose((0, 2, 1))

monster = """\
                  # \
#    ##    ##    ###\
 #  #  #  #  #  #   \
"""
monster = np.array(list(monster)).reshape(3, -1)
monster = monster == "#"

def area_monsters(img, size=15):
    return size * np.sum(convolve(img) == size) or None

def convolve(img, monster=monster[np.newaxis, :, :], h=3, w=20):
    m, n = img.shape
    shifts = product(range(m - h + 1), range(n - w + 1))
    prod = monster & np.array([img[i:i + h, j:j + w] for i, j in shifts])
    return np.sum(prod, axis=(1, 2))

def choppywater(tiles):
    img0 = layout(tiles)
    imgs = accumulate(transform, lambda img, t: t(img), initial=img0)
    return np.sum(img0) - nonnone(map(area_monsters, imgs))

turn, flip = cc[1], lambda x: x[::-1]
transform = [turn, turn, turn, flip, turn, turn ,turn]

assert 2495 == choppywater(tiles)